In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from utils.file_picker import pick_image_cv2

# =========================
# 1. Load images
# =========================

# Full wheel frame (target image)
wheel_bgr = cv2.imread("C:\\Users\\Gabriel\\Documents\\Dissertation\\Code\\data\\roulette2.png")

# Template image
template_bgr, template_path = pick_image_cv2(title="Select a template image")

if wheel_bgr is None or template_bgr is None:
    raise ValueError("Check your file paths – one of the images did not load.")

# Convert to grayscale (matchTemplate works well on grayscale)
wheel_gray = cv2.cvtColor(wheel_bgr, cv2.COLOR_BGR2GRAY)
template_gray = cv2.cvtColor(template_bgr, cv2.COLOR_BGR2GRAY)

h, w = template_gray.shape[:2]

print(f"Template size: {w}x{h}")

In [ ]:
# =========================
# 2. Run template matching for ALL rotated templates
# =========================

# Folder where your rotated templates are saved
templates_dir = "C:\\Users\\Gabriel\\Documents\\Dissertation\\Code\\data\\3pocket3_rotated"

template_paths = sorted(glob.glob(os.path.join(templates_dir, "*.png")))
print(f"Found {len(template_paths)} templates")

if not template_paths:
    raise ValueError("No rotated templates found. Check templates_dir.")

method = cv2.TM_CCOEFF_NORMED  # keep as before

best = {
    "score": -1.0,
    "path": None,
    "top_left": None,
    "w": None,
    "h": None,
    "result": None,
    "template_bgr": None,
}

for tpath in template_paths:
    tmpl_bgr = cv2.imread(tpath)
    tmpl_gray = cv2.cvtColor(tmpl_bgr, cv2.COLOR_BGR2GRAY)

    h, w = tmpl_gray.shape[:2]

    res = cv2.matchTemplate(wheel_gray, tmpl_gray, method)
    _, max_val, _, max_loc = cv2.minMaxLoc(res)

    # Track best overall
    if max_val > best["score"]:
        best.update({
            "score": max_val,
            "path": tpath,
            "top_left": max_loc,
            "w": w,
            "h": h,
            "result": res,
            "template_bgr": tmpl_bgr.copy(),
        })

        print(f"New best: {max_val:.3f} from {os.path.basename(tpath)}")

print("\n=== Final best match ===")
print(f"Template: {best['path']}")
print(f"Score:    {best['score']:.4f}")


In [ ]:
# =========================
# 3. Visualise BEST match (across all rotated templates)
# =========================

top_left = best["top_left"]
w, h = best["w"], best["h"]
bottom_right = (top_left[0] + w, top_left[1] + h)

wheel_vis = wheel_bgr.copy()
cv2.rectangle(wheel_vis, top_left, bottom_right, (0, 255, 0), 2)

wheel_vis_rgb = cv2.cvtColor(wheel_vis, cv2.COLOR_BGR2RGB)
template_rgb = cv2.cvtColor(best["template_bgr"], cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.imshow(template_rgb)
plt.title(f"Best Template\n{os.path.basename(best['path'])}")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(wheel_vis_rgb)
plt.title(f"Best Match (score={best['score']:.3f})")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(best["result"], cmap="hot")
plt.title("Match Heatmap")
plt.colorbar(fraction=0.046, pad=0.04)
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# =========================
# 4. Multiple matches above a threshold (for BEST template)
# =========================
threshold = 0.6
result = best["result"]
w, h = best["w"], best["h"]

if method in [cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR_NORMED]:
    loc = np.where(result >= threshold)
    wheel_multi = wheel_bgr.copy()

    for pt_y, pt_x in zip(*loc):
        top_left_multi = (pt_x, pt_y)
        bottom_right_multi = (pt_x + w, pt_y + h)
        cv2.rectangle(wheel_multi, top_left_multi, bottom_right_multi, (255, 0, 0), 1)

    wheel_multi_rgb = cv2.cvtColor(wheel_multi, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 6))
    plt.imshow(wheel_multi_rgb)
    plt.title(f"Matches with score >= {threshold}")
    plt.axis("off")
    plt.show()
